You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the 'datasets' folder:
- __Office addresses__
    - Saved in `office_addresses.csv`. 
    - If the value for office is `NaN`, then the employee is remote.
- __Employee addresses__
    - Saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ 
    - Saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. 
    - However, this sheet was edited at some point, and ***the headers were removed***! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ 
    - This information has been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
```

Read in, merge, and clean the four datasets to make a single combined pandas DataFrame.

- Create a single pandas DataFrame called employees_final containing:
  - Index: employee_id.
  - Columns: Ensure the DataFrame contains only the following columns, in the exact order listed: employee_first_name, employee_last_name, employee_country, employee_city, employee_street, employee_street_number, emergency_contact, emergency_contact_number, relationship, monthly_salary, team, title, office, office_country, office_city, office_street, office_street_number.
  - Assign employees to offices based on their country. For any columns that begin with office, replace missing data with "Remote".

In [ ]:
import pandas as pd

offices = pd.read_csv("datasets/office_addresses.csv")
addresses = pd.read_excel("datasets/employee_information.xlsx")


offices.head()

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [ ]:
addresses.head()

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [5]:
emergency_contacts_header = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship']

emergency_contacts = pd.read_excel("datasets/employee_information.xlsx", 
                                   sheet_name="emergency_contacts", 
                                   header=None,
                                   names=emergency_contacts_header)

emergency_contacts.head()

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [8]:
roles = pd.read_json("datasets/employee_roles.json")
roles.head()

,A2R5H9,H8K0L6,G4R7V0,M1Z7U9
title,CEO,CFO,Business Developer,Office Manager
monthly_salary,$4500,$4500,$3000,$2000
team,Leadership,Leadership,Sales,People Operations


### Mergowanie

In [9]:
employees = pd.merge(addresses, offices, left_on='employee_country', right_on='office_country', how='left')
employees = pd.merge(employees, emergency_contacts, on='employee_id')
employees = pd.merge(employees, roles, left_on='employee_id', right_on=roles.index)


### Zamiana brakujących wartości w kolumnach zaczynających się na 'office'

In [12]:
for col in ["office", "office_country", "office_city", "office_street", "office_street_number"]:
    employees.fillna({col: 'Remote'}, inplace=True)

### Utworzenie finalnego df:

In [ ]:
final_columns = ["employee_id", "employee_first_name", "employee_last_name", "employee_country", 
                 "employee_city", "employee_street", "employee_street_number", 
                 "emergency_contact", "emergency_contact_number", "relationship", 
                 "monthly_salary", "team", "title", "office", "office_country", 
                 "office_city", "office_street", "office_street_number"]

employees_final = employees[final_columns]
employees_final

# Solution

In [ ]:
# Import pandas
import pandas as pd

# Read in office_addresses.csv
offices = pd.read_csv("datasets/office_addresses.csv")

# Read in employee_information.xlsx
addresses = pd.read_excel("datasets/employee_information.xlsx")

# Declare a list of new column names
emergency_contacts_header = ["employee_id", "last_name", "first_name",
                             "emergency_contact", "emergency_contact_number", "relationship"]

# Read in employee_information.xlsx
emergency_contacts = pd.read_excel("datasets/employee_information.xlsx", 
                                   sheet_name="emergency_contacts", 
                                   header=None,
                                   names=emergency_contacts_header)

# Read in employee_roles.json
roles = pd.read_json("datasets/employee_roles.json", orient="index")

# Merge addresses with offices
employees = addresses.merge(offices, left_on="employee_country", right_on="office_country", how="left")

# Merge employees with roles
employees = employees.merge(roles, left_on="employee_id", right_on=roles.index)

# Merge employees with emergency_contacts
employees = employees.merge(emergency_contacts, on="employee_id")

# Fill null values in office columns
for col in ["office", "office_country", "office_city", "office_street", "office_street_number"]:
    employees[col].fillna("Remote", inplace=True)
    
# Create final columns
final_columns = ["employee_id", "employee_first_name", "employee_last_name", "employee_country", 
                 "employee_city", "employee_street", "employee_street_number", 
                 "emergency_contact", "emergency_contact_number", "relationship", 
                 "monthly_salary", "team", "title", "office", "office_country", 
                 "office_city", "office_street", "office_street_number"]

# Subset for the required columns
employees_final = employees[final_columns]

# Set employee_id as the index
employees_final.set_index("employee_id", inplace=True)